In [81]:
# will autoupdate any of the packages imported:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from numba import njit

In [2]:
import sys
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
import pyclifford as pc
from pyclifford import utils, paulialg, stabilizer, circuit
from pyclifford.utils import acq, ipow, clifford_rotate, pauli_transform
from pyclifford.paulialg import Pauli, PauliList, PauliPolynomial
import qutip as qt

In [3]:
from pyclifford.paulialg import pauli, Pauli

In [4]:
pauli("YY")+pauli("XY")+pauli("YY")

1 XY +2 YY

Backend function:

In [21]:
@njit
def decompose(g, gs, ps):
    '''  Decompose a pauli string to phase*destabilizers*stabilizers
    Parameters:
    g: int(2*N) - the binary vector of a pauli string
    gs: int(2*N,2*N) - the full tableau
    ps: int(2*N) - phase of stabilizer and destabilizer
    
    Returns:
    phase: int - phase in terms of imaginery power
    b: int(N) - binary encoding of destabilizer decomposition
    c: int(N) - binary encoding of stabilizer decomposition
    '''
    phase = 0
    tmp_p = np.zeros_like(g)
    N = gs.shape[0]//2
    # b = int(np.zeros(N)) # numbda does not support change data type
    # c = int(np.zeros(N))
    b = np.array([0 for _ in range(N)])
    c = np.array([0 for _ in range(N)])
    for i in range(N):
        if acq(g,gs[i]): #anti-commute
            b[i] = 1
            phase = phase - ipow(tmp_p,gs[i+N]) + ps[i+N]
            tmp_p = (tmp_p+gs[i+N])%2
    for i in range(N):
        if acq(g,gs[i+N]): #anti-commute
            c[i] = 1
            phase = phase - ipow(tmp_p,gs[i]) + ps[i]
            tmp_p = (tmp_p+gs[i])%2
    return phase%4, tmp_p, b, c

The above function will decompose a pauli string into combination of destabilizer generators and stabilizer generators.

Let's see an example, if the stabilizer generators are $g_0=-ZZ$ and $g_1=XX$ and destabilizer generators are $d_0=IX$, $d_1=ZI$.

In [24]:
gs = stabilizer.stabilizer_state("-ZZ","XX").gs
ps = stabilizer.stabilizer_state("-ZZ","XX").ps

In [25]:
decompose(np.array([1,0,1,1]),gs,ps)

(1, array([1, 0, 1, 1]), array([0, 1]), array([1, 1]))

From the result, we see pauli string $XY$ can be decomposed as
$$XY = i^1 d_0^0 d_1^1 g_0^1 g_1^1=i(XI)(-ZZ)(XX)$$

# General Stabilizer State

In [49]:
class GeneralStabilizerState(object):

    def __init__(self, chi, gs, ps):
        self.chi = chi
        self.gs = gs
        self.ps = ps
        self.n = gs.shape[1]//2
        
    def copy(self):
        return self.chi.copy(), self.gs.copy(), self.ps.copy()

    def rotate_by(self, generator, mask=None):
        # perform Clifford transformation by Pauli generator (in-place)
        if mask is None:
            clifford_rotate(generator.g, generator.p, self.gs, self.ps)
        else:
            mask2 = numpy.repeat(mask,  2)
            self.gs[:,mask2], self.ps = clifford_rotate(
                generator.g, generator.p, self.gs[:,mask2], self.ps)
        return self
        
    def transform_by(self, clifford_map, mask=None):
        # perform Clifford transformation by Clifford map (in-place)
        if mask is None:
            self.gs, self.ps = pauli_transform(self.gs, self.ps, 
                clifford_map.gs, clifford_map.ps)
        else:
            mask2 = numpy.repeat(mask, 2)
            self.gs[:,mask2], self.ps = pauli_transform(
                self.gs[:,mask2], self.ps, clifford_map.gs, clifford_map.ps)
        return self
    
    def pauli_chnl_evol(self, gate):
        '''Perform general Clifford channel evolution.
        
        Parameters:
        phi: [c1, c2, ...] - list of Pauli channel coefficients.
        pl:  [int (2*N), int (2*N), ...] - list of left-multiplying Pauli ops.
        pr:  [int (2*N), int (2*N), ...] - list of right-multiplying Pauli ops. 
        
        Returns:
        chi in-place modified.
        '''
        # pre-store alpha, b, c to avoid redundant call of decompose
        phi, pl, pr = gate.phi, gate.pl, gate.pr
        al, bl, cl = [], [], []
        ar, br, cr = [], [], []
        for pm, pn in zip(pl, pr):
            am, _, bm, cm = decompose(pm, self.gs, self.ps) 
            an, _, bn, cn = decompose(pn, self.gs, self.ps)
            al.append(am)
            bl.append(bm)
            cl.append(cm)
            ar.append(an)
            br.append(bn)
            cr.append(cn)
        # update chi
        chinew = {}
        for idx in range(len(phi)):
            phimn = phi[idx]
            am, bm, cm = al[idx], bl[idx], cl[idx]
            an, bn, cn = ar[idx], br[idx], cr[idx]
            # print(phimn)
            for (i, j), chiij in self.chi.items():
                i, j = np.array(list(i)), np.array(list(j))
                # update i, j
                inew = (i + bm)%2
                jnew = (j + bn)%2
                # update chiij
                ipow = (am-an+2*(np.dot(i,cm)+np.dot(j,cn)))%4
                chiijnew = chiij * phimn * 1j**ipow
                keynew = (tuple(inew), tuple(jnew))
                if keynew in chinew:
                    chinew[keynew] += chiijnew
                else:
                    chinew[keynew] = chiijnew
                # print(chinew)
        self.chi = chinew         
    def pauli_expect(self, obs):
        '''Evaluate expectation values of Pauli observables on the
generalized stabilizer state.
        
        Parameters:
        obs: observable, can be Pauli, PauliPolynomial, PauliList
        z: fugacity of operator weight
        
        Returns:
        out: output (depending on the type of obs)
            * Pauli: promote to PauliPolynomial
            * PauliPolynomial O: Tr(rho O z^|O|)
            * Paulilist [O_i]: [Tr(rho O_i z^|O_i|)]
        '''
        if isinstance(obs, Pauli):
            # cast Pauli to PauliPolynomial
            return self.pauli_expect(obs.as_polynomial())
        elif isinstance(obs, PauliPolynomial):
            # cast PauliPolynomial to PauliList
            xs = self.expect(PauliList(obs.gs, obs.ps))
            return numpy.sum(obs.cs * xs)
        elif isinstance(obs, PauliList):
            (L, Ng) = obs.gs.shape
            N = Ng//2
            xs = np.zeros(L, dtype=np.complex_) # expectation values
            pa = 0
            for k in range(L):
                a, _, b, c = decompose(obs.gs[k], self.gs, self.ps)
                for (i, j), chi in self.chi.items():
                    i, j = np.array(list(i)), np.array(list(j))
                    # print(f'i: {i.shape}, j: {j.shape}, b:{b.shape}')
                if np.all((i+j+b) == 0):
                    # ipow = (a + obs.ps[k] + 2*i@c) % 4
                    ipow = (a + obs.ps[k] + 2*np.dot(i, c)) % 4
                    xs[k] += chiij * 1j**ipow
            return xs 
    def to_qutip(self):
        n = self.gs.shape[1]//2
        paulis = [qt.qeye(2), qt.sigmax(), qt.sigmay(), qt.sigmaz()]
        identity = qt.tensor([qt.qeye(2) for i in range(n)])
        stab_state = identity
        for i in range(n):
            current_stabilizer = pc.paulialg.Pauli(g=self.gs[i],p=self.ps[i]).to_qutip()
            stab_state *= (identity+current_stabilizer)/2
            # print('current_stabilizer:',current_stabilizer, 'stab_state:', stab_state)
        state = 0
        for k, e in self.chi.items():
            left_coor = k[0]
            right_coor = k[1]
            # print('left_coor:',left_coor,'right_coor',right_coor)
            left_destabilizer = identity
            for l in range(len(left_coor)):
                if left_coor[l] == 1:
                    left_destabilizer *= pc.paulialg.Pauli(g=self.gs[l+n],p=self.ps[l+n]).to_qutip()
                    # print(f'{l}th left destabilizer: {left_destabilizer}')
            right_destabilizer = identity
            for r in range(len(right_coor)):
                if right_coor[r] == 1:
                    right_destabilizer *= pc.paulialg.Pauli(g=self.gs[r+n],p=self.ps[r+n]).to_qutip()
                    # print(f'{r}th right destabilizer: {left_destabilizer}')
            state += e * left_destabilizer * stab_state * right_destabilizer
        return state

In [50]:
class GeneralGate(object):
    def __init__(self, acting_qubit, num_qubits, phi, pl, pr):
        '''
        General gate U can be represented as U^{dagger}rho U = \sum_{ij} phi_{ij} P_i \rho P_j
        '''
        self.phi = phi
        self.pl = pl
        self.pr = pr
        self.i = acting_qubit
        self.n = num_qubits

In [51]:
np.zeros(3).astype(int)

array([0, 0, 0])

In [52]:
def Tgate(i,n):
    '''
    Parameters:
    i - int: which qubit this gate is acting on
    n - int: total number of qubits in the system
    
    Returns:
    gate - GeneralGate
    '''
    phi = np.array([np.cos(np.pi/8)**2, 1j*np.sin(np.pi/8)*np.cos(np.pi/8),\
                -1j*np.sin(np.pi/8)*np.cos(np.pi/8), np.sin(np.pi/8)**2], dtype=np.complex_)
    extend_I = np.zeros(2*n).astype(int)
    extend_Z = np.zeros(2*n).astype(int)
    extend_Z[2*i+1]=1
    pl = [extend_I, extend_Z, extend_I, extend_Z]
    pr = [extend_I, extend_I, extend_Z, extend_Z]
    return GeneralGate(i,n,phi,pl,pr)

In [58]:
T0 = Tgate(0,3)
T2 = Tgate(2,3)

In [63]:
{(tuple(0 for i in range(3)),tuple(0 for i in range(3))):1.0}

{((0, 0, 0), (0, 0, 0)): 1.0}

In [59]:
ghz = GeneralStabilizerState({(tuple(0 for i in range(3)),tuple(0 for i in range(3))):1.0},\
                            gs = pc.ghz_state(3).gs,ps = pc.ghz_state(3).ps)

In [61]:
ghz.pauli_chnl_evol(T0)
ghz.pauli_chnl_evol(T2)

In [62]:
ghz.to_qutip()

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.5+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.5j]
 [0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j ]
 [0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j ]
 [0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j ]
 [0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j ]
 [0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j ]
 [0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j ]
 [0. -0.5j 0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0.5+0.j ]]